In [1]:
# Import appropriate modules from the client library.
from googleads import dfp
from collections import defaultdict
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()

sc =SparkContext()
sqlCtx = SQLContext(sc)


#connect to the dfp network
from googleads import dfp
dfp_client = dfp.DfpClient.LoadFromStorage()
network_service = dfp_client.GetService('NetworkService', version='v201708')
current_network = network_service.getCurrentNetwork()


In [73]:
line_items = sqlCtx.read.parquet("gs://ds-url-catag/dfp_new/line_items/")

In [74]:
line_items = line_items.withColumn("start_date",concat(col('start_date_year'),lit('-'),col('start_date_month'),lit('-'),col('start_date_day')))
line_items = line_items.withColumn("End_date",concat(col('End_date_year'),lit('-'),col('End_date_month'),lit('-'),col('End_date_day')))

#dropping columns in the line items dataset
drop_list = ['start_date_year','start_date_month','start_date_day','end_date_year','end_date_month','end_date_day']
new_line_items = line_items.select([column for column in line_items.columns if column not in drop_list])

In [75]:
#reading custom values high level dataset
custom_values_high = sqlCtx.read.parquet("gs://ds-url-catag/dfp_new/custom_fields_line_high/")

#joinig with highlevel custom fields to find the type of demographics
tier1data = new_line_items.join(custom_values_high,custom_values_high.id == new_line_items.target_demo,'left')


# #dropping unwanted columns
# drop_list = ['target_demo','id']
# tier2data = tier1data.select([column for column in tier1data.columns if column not in drop_list])

In [76]:
#transposing rows to columns
new_line_items_piv = tier1data.groupBy("line_item_id","order_id","line_item_name","budget","currency_code","impressions","clicks","target_country","target_os","target_audienceid","start_date","End_date").pivot("displayname").agg(expr("coalesce(first(values))"))


In [77]:
drop_list = ['Premium Slot','SV&V Test Targeting Flag','YourBow Pre-Flight Flag','bbm_channel','channel','staging_bbm_channel','null','Capabilities']
tier2data = new_line_items_piv.select([column for column in new_line_items_piv.columns if column not in drop_list])

In [79]:
from pyspark.sql.functions import split
import re
#cleaning the values column Ad Type
def trimming_brackets(s):
    final_string = ''
    if s is not None :
        new_string = s.strip('[')
        new_string = new_string.strip(']')
    else:
        new_string = None
    return new_string

filter_udf = udf(trimming_brackets, StringType())

tier2data = tier2data.withColumn('Ad Type',filter_udf(col("Ad Type")))


#Splitting rows to columns for Ad Type

tier2data = tier2data.withColumn('ad_Type_1',split(col('Ad Type'),',')[0])
tier2data = tier2data.withColumn('ad_Type_1',regexp_replace(col('ad_Type_1'),'L',''))
# tier2data = tier2data.withColumn('ad_Type_1',trim(col('ad_Type_1')))
tier2data = tier2data.withColumn('ad_Type_2',split(col('Ad Type'),',')[1])
tier2data = tier2data.withColumn('ad_Type_2',regexp_replace(col('ad_Type_2'),'L',''))
tier2data = tier2data.withColumn('ad_Type_2',trim(col('ad_Type_2')))
tier2data = tier2data.withColumn('ad_Type_3',split(col('Ad Type'),',')[2])
tier2data = tier2data.withColumn('ad_Type_3',regexp_replace(col('ad_Type_3'),'L',''))
tier2data = tier2data.withColumn('ad_Type_3',trim(col('ad_Type_3')))
tier2data = tier2data.withColumn('ad_Type_4',split(col('Ad Type'),',')[3])
tier2data = tier2data.withColumn('ad_Type_4',regexp_replace(col('ad_Type_4'),'L',''))
tier2data = tier2data.withColumn('ad_Type_4',trim(col('ad_Type_4')))

drop_list = ['Ad Type']
tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

In [80]:
#cleaning the values column Age group
filter_udf = udf(trimming_brackets, StringType())

tier2data = tier2data.withColumn('Age Group',filter_udf(col("Age Group")))


#Splitting rows to columns for Ad Type
tier2data = tier2data.withColumn('Age_group_1',split(col('Age Group'),',')[0])
tier2data = tier2data.withColumn('Age_group_1',regexp_replace(col('Age_group_1'),'L',''))
tier2data = tier2data.withColumn('Age_group_1',trim(col('Age_group_1')))
tier2data = tier2data.withColumn('Age_group_2',split(col('Age Group'),',')[1])
tier2data = tier2data.withColumn('Age_group_2',regexp_replace(col('Age_group_2'),'L',''))
tier2data = tier2data.withColumn('Age_group_2',trim(col('Age_group_2')))
tier2data = tier2data.withColumn('Age_group_3',split(col('Age Group'),',')[2])
tier2data = tier2data.withColumn('Age_group_3',regexp_replace(col('Age_group_3'),'L',''))
tier2data = tier2data.withColumn('Age_group_3',trim(col('Age_group_3')))
tier2data = tier2data.withColumn('Age_group_4',split(col('Age Group'),',')[3])
tier2data = tier2data.withColumn('Age_group_4',regexp_replace(col('Age_group_4'),'L',''))
tier2data = tier2data.withColumn('Age_group_4',trim(col('Age_group_4')))
tier2data = tier2data.withColumn('Age_group_5',split(col('Age Group'),',')[4])
tier2data = tier2data.withColumn('Age_group_5',regexp_replace(col('Age_group_5'),'L',''))
tier2data = tier2data.withColumn('Age_group_5',trim(col('Age_group_5')))
tier2data = tier2data.withColumn('Age_group_6',split(col('Age Group'),',')[5])
tier2data = tier2data.withColumn('Age_group_6',regexp_replace(col('Age_group_6'),'L',''))
tier2data = tier2data.withColumn('Age_group_6',trim(col('Age_group_6')))
tier2data = tier2data.withColumn('Age_group_7',split(col('Age Group'),',')[6])
tier2data = tier2data.withColumn('Age_group_7',regexp_replace(col('Age_group_7'),'L',''))
tier2data = tier2data.withColumn('Age_group_7',trim(col('Age_group_7')))
tier2data = tier2data.withColumn('Age_group_8',split(col('Age Group'),',')[7])
tier2data = tier2data.withColumn('Age_group_8',regexp_replace(col('Age_group_8'),'L',''))
tier2data = tier2data.withColumn('Age_group_8',trim(col('Age_group_8')))
 

drop_list = ['Age Group']
tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])
        

In [81]:
#cleaning the values column channels
tier2data = tier2data.withColumn('Channels',filter_udf(col("Channels")))


#Splitting rows to columns for Ad Type
tier2data = tier2data.withColumn('channels_1',split(col('Channels'),',')[0])
tier2data = tier2data.withColumn('channels_1',regexp_replace(col('channels_1'),'L',''))
tier2data = tier2data.withColumn('channels_1',trim(col('channels_1')))
tier2data = tier2data.withColumn('channels_2',split(col('Channels'),',')[1])
tier2data = tier2data.withColumn('channels_2',regexp_replace(col('channels_2'),'L',''))
tier2data = tier2data.withColumn('channels_2',trim(col('channels_2')))

drop_list = ['Channels']
tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

In [82]:
#cleaning the values column Gender
tier2data = tier2data.withColumn('Gender',filter_udf(col("Gender")))
tier2data = tier2data.withColumn('Gender',regexp_replace(col('Gender'),'L',''))
tier2data = tier2data.withColumn('Gender',trim(col('Gender')))



In [83]:
#cleaning the values column Gender
tier2data = tier2data.withColumn('target_audienceid',filter_udf(col("target_audienceid")))
tier2data = tier2data.withColumn('target_audienceid',regexp_replace(col('target_audienceid'),'L',''))
tier2data = tier2data.withColumn('target_audienceid',trim(col('target_audienceid')))


In [84]:
#reading custom fields low
custom_values_low = sqlCtx.read.parquet("gs://ds-url-catag/dfp_new/custom_fields_line/")

#renamingcolumns 
custom_values_low = custom_values_low.withColumn("or_displayname",col('displayname'))
custom_values_low = custom_values_low.withColumn("or_customtargetid",col('customtargetid'))

#dropping unwanted columns
drop_list = ['displayname','customtargetid']
custom_values_low = custom_values_low.select([column for column in custom_values_low.columns if column not in drop_list])


In [85]:
# #joinig Adtype to get the values
#adtye 1
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.ad_Type_1,'left')
tier2data =tier2data.withColumn('ad_Type_1',col('name'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])


#adtye 2
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.ad_Type_2,'left')
tier2data =tier2data.withColumn('ad_Type_2',col('name'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

#adtye 3
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.ad_Type_3,'left')
tier2data =tier2data.withColumn('ad_Type_3',col('name'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

#adtye 4
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.ad_Type_4,'left')
tier2data =tier2data.withColumn('ad_Type_4',col('name'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])



In [86]:
# #joinig Age group to get the values
#Agegroup1
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_1,'left')
tier2data =tier2data.withColumn('Age_group_1',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])


#Agegroup2
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_2,'left')
tier2data =tier2data.withColumn('Age_group_2',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

#Agegroup3
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_3,'left')
tier2data =tier2data.withColumn('Age_group_3',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

#Agegroup4
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_4,'left')
tier2data =tier2data.withColumn('Age_group_4',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

#Agegroup5
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_5,'left')
tier2data =tier2data.withColumn('Age_group_5',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])


#Agegroup6
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_6,'left')
tier2data =tier2data.withColumn('Age_group_6',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

#Agegroup7
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_7,'left')
tier2data =tier2data.withColumn('Age_group_7',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

#Agegroup8
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Age_group_8,'left')
tier2data =tier2data.withColumn('Age_group_8',col('or_displayname'))
drop_list = ['id','name','or_displayname','or_customtargetid']

tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

In [87]:
# #joinig Gender to get the values
#Agegroup1
tier2data = tier2data.join(custom_values_low,custom_values_low.id == tier2data.Gender,'left')
tier2data =tier2data.withColumn('Gender',col('name'))
drop_list = ['id','name','or_displayname','or_customtargetid']
tier2data = tier2data.select([column for column in tier2data.columns if column not in drop_list])

In [89]:
tier3data = tier2data.groupby('line_item_id','order_id','line_item_name','budget','currency_code','impressions','clicks','target_country','target_os','start_date','End_date').agg(max('Gender').alias('Gender'),max("ad_type_1").alias('ad_type_1'),max("ad_type_2").alias('ad_type_2'),max("ad_type_3").alias('ad_type_3'),max("ad_type_4").alias('ad_type_4'),max("Age_group_1").alias('Age_group_1'),max("Age_group_2").alias('Age_group_2'),max("Age_group_3").alias('Age_group_3'),max("Age_group_4").alias('Age_group_4'),max("Age_group_5").alias('Age_group_5'),max("Age_group_6").alias('Age_group_6'),max("Age_group_7").alias('Age_group_7'),max("Age_group_8").alias('Age_group_8'),max("channels_1").alias('channels_1'),max("channels_2").alias('channels_2'),max("target_audienceid").alias('target_audienceid'))

In [94]:
tier3data.write.mode('overwrite').parquet("gs://ds-url-catag/campaign_analysis/processed_orders/")